In [1]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [2]:
sitemaps = pd.read_xml("https://www.hln.be/sitemap-news.xml")
sitemaps.head()

,loc,lastmod,news,image
0,https://www.hln.be/de-panne/zonder-haar-mama-z...,2023-08-02T09:06:08+00:00,NaN,NaN
1,https://www.hln.be/dentergem/ruim-400-senioren...,2023-08-02T09:06:03+00:00,NaN,NaN
2,https://www.hln.be/mechelen/bromfietser-73-gew...,2023-08-02T09:05:00+00:00,NaN,NaN
3,https://www.hln.be/wetteren/nieuw-breek-breekt...,2023-08-02T09:04:26+00:00,NaN,NaN
4,https://www.hln.be/dendermonde/agentschap-wege...,2023-08-02T09:04:11+00:00,NaN,NaN


In [3]:
urls = sitemaps["loc"]
print(urls)

0       https://www.hln.be/de-panne/zonder-haar-mama-z...
1       https://www.hln.be/dentergem/ruim-400-senioren...
2       https://www.hln.be/mechelen/bromfietser-73-gew...
3       https://www.hln.be/wetteren/nieuw-breek-breekt...
4       https://www.hln.be/dendermonde/agentschap-wege...
                              ...                        
1272    https://www.hln.be/lievegem/waarschoots-kampio...
1273    https://www.hln.be/nieuwpoort/voor-10-euro-kan...
1274    https://www.hln.be/binnenland/debat-van-de-dag...
1275    https://www.hln.be/kasterlee/100-retro-festiva...
1276    https://www.hln.be/riemst/maastricht-umc-overw...
Name: loc, Length: 1277, dtype: object


In [4]:
url = urls[0]
response = requests.get(url).text
print(response)


<!DOCTYPE html>
<html lang='nl'>
<head>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">

    <style>
        @font-face {
            font-family: Sharp Sans Display;
            font-style: normal;
            font-weight: 600;
            /*noinspection CssUnknownTarget*/
            src: url("https://myprivacy-static.dpgmedia.net/consent/resources/fonts/SharpSansDispNo2-Bold.otf") format("opentype");
        }

        @font-face {
            font-family: Montserrat;
            font-style: normal;
            font-weight: 400;
            /*noinspection CssUnknownTarget*/
            src: url("https://myprivacy-static.dpgmedia.net/consent/resources/fonts/Montserrat-Regular.ttf") format("opentype");
        }

        h2 {
            font-family: Sharp Sans Display, Arial, sans-serif;
            font-size: 26px;
            text-transform: uppercase;
        }

        body {
            margin: 0;
            font-family: Montserrat, Arial, sans-serif

In [7]:
soup = BeautifulSoup(response, "html.parser")
print(soup)


<!DOCTYPE html>

<html lang="nl">
<head>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<style>
        @font-face {
            font-family: Sharp Sans Display;
            font-style: normal;
            font-weight: 600;
            /*noinspection CssUnknownTarget*/
            src: url("https://myprivacy-static.dpgmedia.net/consent/resources/fonts/SharpSansDispNo2-Bold.otf") format("opentype");
        }

        @font-face {
            font-family: Montserrat;
            font-style: normal;
            font-weight: 400;
            /*noinspection CssUnknownTarget*/
            src: url("https://myprivacy-static.dpgmedia.net/consent/resources/fonts/Montserrat-Regular.ttf") format("opentype");
        }

        h2 {
            font-family: Sharp Sans Display, Arial, sans-serif;
            font-size: 26px;
            text-transform: uppercase;
        }

        body {
            margin: 0;
            font-family: Montserrat, Arial, sans-serif;
     

In [6]:
title = soup.find("h1").text
print(title)

AttributeError: 'NoneType' object has no attribute 'text'

In [8]:
article = soup.find_all("p", attrs={"class": None})
articles = [p.text for p in article]
cleaned_articles = " ".join(articles)
print(cleaned_articles)

In [9]:
script = soup.find("script", {"type": "application/ld+json"})
data = json.loads(script.text, strict=False)

accessing_list = data["@graph"]
accessing_dict = accessing_list[0]
published_date = accessing_dict["datePublished"]
# accessing_list2 = data.get("@graph",[{}])[0].get("datePublished", None)
# print(accessing_list)
# print(accessing_dict)
print(published_date)
# print(accessing_list2)

AttributeError: 'NoneType' object has no attribute 'text'